In [1]:
import os
from qiskit.circuit.library import EfficientSU2
import math
import cirq
import numpy as np
import openfermion as of
#import stim
#import stimcirq
import h5py
from typing import Set, List, Iterable
import warnings
from qiskit_aer.noise import NoiseModel, pauli_error, depolarizing_error
from qiskit import transpile
import re
from qiskit.quantum_info import SparsePauliOp
from qiskit.quantum_info import Operator

# Configure module paths
import sys
sys.path.insert(1, "_common")
sys.path.insert(1, "qiskit")

from hamlib_utils import load_from_file, get_hamlib_sparsepaulilist 
from hamlib_simulation_kernel import ensure_sparse_pauli_op

from qiskit import QuantumCircuit
from cirq import IdentityGate
from qiskit_aer import AerSimulator
from cirq.ops.dense_pauli_string import DensePauliString
from qiskit_ibm_runtime import EstimatorV2 as Estimator

In [6]:
# 'H2': '/ham_BK-4'

hamiltonian_name = 'chemistry/electronic/standard/H2'
hamiltonian_params = { "ham_BK": '' }
num_qubits = 4

# load the HamLib file for the given hamiltonian name
load_from_file(filename=hamiltonian_name)

# return a sparse Pauli list of terms queried from the open HamLib file
sparse_pauli_terms, dataset_name = get_hamlib_sparsepaulilist(num_qubits=num_qubits, params=hamiltonian_params)
print(f"... sparse_pauli_terms = \n{sparse_pauli_terms}")

# convert the SparsePauliList to a SparsePauliOp object
sparse_pauli_op = ensure_sparse_pauli_op(sparse_pauli_terms, num_qubits)
print(f"... sparse_pauli_op = \n{sparse_pauli_op}")
print("")


... sparse_pauli_terms = 
[({}, (-0.44779757933958947+0j)), ({0: 'X', 1: 'Z', 2: 'X'}, (0.014034099995004047+0j)), ({0: 'X', 1: 'Z', 2: 'X', 3: 'Z'}, (0.014034099995004047+0j)), ({0: 'Y', 1: 'Z', 2: 'Y'}, (0.014034099995004047+0j)), ({0: 'Y', 1: 'Z', 2: 'Y', 3: 'Z'}, (0.014034099995004047+0j)), ({0: 'Z'}, (0.2823508885110738+0j)), ({0: 'Z', 1: 'Z'}, (0.28235088851107354+0j)), ({0: 'Z', 1: 'Z', 2: 'Z'}, (0.09615022483162383+0j)), ({0: 'Z', 1: 'Z', 2: 'Z', 3: 'Z'}, (0.09615022483162383+0j)), ({0: 'Z', 2: 'Z'}, (0.08211612483661979+0j)), ({0: 'Z', 2: 'Z', 3: 'Z'}, (0.08211612483661979+0j)), ({1: 'Z'}, (0.16462320552994025+0j)), ({1: 'Z', 2: 'Z', 3: 'Z'}, (-0.0039867476924421025+0j)), ({1: 'Z', 3: 'Z'}, (0.08366738652351666+0j)), ({2: 'Z'}, (-0.003986747692442089+0j))]
... sparse_pauli_op = 
SparsePauliOp(['IIII', 'XZXI', 'XZXZ', 'YZYI', 'YZYZ', 'ZIII', 'ZZII', 'ZZZI', 'ZZZZ', 'ZIZI', 'ZIZZ', 'IZII', 'IZZZ', 'IZIZ', 'IIZI'],
              coeffs=[-0.448+0.j,  0.014+0.j,  0.014+0.j,  0.014+

In [ ]:
paulis = {
        'X' : np.array([[0, 1], [1, 0]]),
        'Y': np.array([[0, -1j], [1j, 0]]),
        'Z': np.array([[1, 0], [0, -1]]),
        'I': np.array([[1, 0], [0, 1]])
}
I = np.array([[1, 0], [0, 1]])
Z = np.array([[1, 0], [0, -1]])

In [35]:
def get_expectation_value(term, counts):
    """
    Computes the expectation value of a measurement outcome with respect to a single Pauli operator.

    Args:
        term (str): A string representing a Pauli operator (e.g., 'ZZI', 'ZZI', 'III'), 
                    where each character corresponds to the Pauli operator ('X', 'Y', 'Z', or 'I') 
                    acting on a specific qubit.
        counts (dict): A dictionary containing measurement results as keys (bitstrings) and 
                       their corresponding counts as values.

    Returns:
        float: The expectation value of the measurement results with respect to the specified Pauli term.
    """
    exp_val = 0
    total_counts = sum(counts.values())  # Total number of measurement shots
    num_qubits = len(term)  # Total number of qubits in the system

    # Loop over all measurement results
    for bitstring, count in counts.items():
        parity = 1.0  # Initialize parity for the current bitstring

        # Iterate over each qubit and its corresponding Pauli operator
        for qubit_index, pauli in enumerate(term):
            if pauli == 'I':  # Skip identity operators, as they do not affect the parity
                continue

            # Map qubit index to bitstring index (little-endian order) and extract bit value
            bit_index = num_qubits - 1 - qubit_index
            bit_value = int(bitstring[bit_index])

            # Map bit value (0 or 1) to eigenvalue (+1 or -1)
            eigenvalue = 1 - 2 * bit_value
            parity *= eigenvalue  # Update parity based on the eigenvalue

        exp_val += parity * count  # Weighted sum of parities based on counts
    # Normalize by the total number of measurement shots
    return exp_val / total_counts


In [36]:
def observables_old(groupings, qubits):
        """Compute the measurement circuit."""
        old_all_obs = []
        for idx, group in enumerate(groupings):
                observables = []
                for op in group:
                        ps = restrict_to(op, qubits)
                        dps = ps.dense(qubits)
                        observables.append(dps)
                        # print(dps)
                if idx == 0:
                        # add the identity observable
                        identity_key = ''.join('I' for _ in range(len(qubits)))
                        if identity_key in pauli_coeffs.keys():
                                observables.append(DensePauliString(identity_key, coefficient=pauli_coeffs[identity_key]))
                old_all_obs.append(observables)
        return old_all_obs

In [37]:
def read_openfermion_hdf5(fname_hdf5: str, key: str, optype=of.QubitOperator):
        """
        Read any openfermion operator object from HDF5 file at specified key.
        'optype' is the op class, can be of.QubitOperator or of.FermionOperator.
        """

        with h5py.File(fname_hdf5, 'r', libver='latest') as f:
                op = optype(f[key][()].decode("utf-8"))
        return op

In [38]:
def get_hdf5_keys(fname_hdf5: str):
        """ Get a list of keys to all datasets stored in the HDF5 file .
        Args
        ----
        fname_hdf5 ( str ) : full path where HDF5 file is stored
        """

        all_keys = []

        @parse_through_hdf5
        def action(obj, path='/', key=None, leaf=False):
                if leaf is True:
                        all_keys.append(path)

        with h5py.File(fname_hdf5, 'r') as f:
                action(f['/'])
        return all_keys

In [39]:
def parse_through_hdf5(func):
        """
        Decorator function that iterates through an HDF5 file and performs
        the action specified by ‘ func ‘ on the internal and leaf nodes in the HDF5 file.
        """

        def wrapper(obj, path='/', key=None):
                if type(obj) in [h5py._hl.group.Group, h5py._hl.files.File]:
                        for ky in obj.keys():
                                func(obj, path, key=ky, leaf=False)
                                wrapper(obj=obj[ky], path=path + ky + ',', key=ky)
                elif type(obj) == h5py._hl.dataset.Dataset:
                        func(obj, path, key=None, leaf=True)

        return wrapper


In [40]:
def preprocess_hamiltonian(
        hamiltonian: of.QubitOperator,
        drop_term_if=None,
) -> cirq.PauliSum:
        """Drop identity terms from the Hamiltonian and convert to Cirq format.
        """
        if drop_term_if is None:
                drop_term_if = []

        new = cirq.PauliSum()

        for term in hamiltonian.terms:
                add_term = True

                for drop_term in drop_term_if:
                        if drop_term(term):
                                add_term = False
                                break

                if add_term:
                        key = " ".join(pauli + str(index) for index, pauli in term)
                        new += next(iter(of.transforms.qubit_operator_to_pauli_sum(
                                of.QubitOperator(key, hamiltonian.terms.get(term))
                        )))

        return new


In [41]:
def compute_groups(k):
        return get_si_sets(hamiltonian, k)

In [42]:
def get_terms_ordered_by_abscoeff(ham: cirq.PauliSum) -> List[cirq.PauliString]:
        """Returns the terms of the PauliSum ordered by coefficient absolute value.

        Args:
            ham: A PauliSum.
        Returns:
            a list of PauliStrings sorted by the absolute value of their coefficient.
        """
        return sorted([term for term in ham], key=lambda x: abs(x.coefficient), reverse=True)

In [43]:
def get_si_sets(ham: cirq.PauliSum, k: int = 1) -> List[List[cirq.PauliString]]:
        """Returns grouping from the sorted insertion algorithm [https://quantum-journal.org/papers/q-2021-01-20-385/].

        Args:
            op: The observable to group.
            k: The integer k in k-commutativity.
        """

        qubits = sorted(set(ham.qubits))
        blocks = compute_blocks(qubits, k)

        commuting_sets = []
        for pstring in get_terms_ordered_by_abscoeff(ham):
                found_commuting_set = False

                for commset in commuting_sets:
                        cant_add = False

                        for pauli in commset:
                                if not commutes(pstring, pauli, blocks):
                                        cant_add = True
                                        break

                        if not cant_add:
                                commset.append(pstring)
                                found_commuting_set = True
                                break

                if not found_commuting_set:
                        commuting_sets.append([pstring])

        return commuting_sets


In [44]:
def commutes(pauli1: cirq.PauliString, pauli2: cirq.PauliString, blocks) -> bool:
        """Returns True if pauli1 k-commutes with pauli2, else False.

        Arguments:
            pauli1: A Pauli string.
            pauli2: A Pauli string.
            blocks: The block partitioning.

        """

        for block in blocks:
                if not cirq.commutes(restrict_to(pauli1, block), restrict_to(pauli2, block)):
                        return False
        return True


In [45]:
def compute_blocks(qubits, k):
        return [qubits[k * i: k * (i + 1)] for i in range(math.ceil(len(qubits) / k))]

In [46]:
def restrict_to(
        pauli: cirq.PauliString, qubits: Iterable[cirq.Qid]
) -> cirq.PauliString:
        """Returns the Pauli string restricted to the provided qubits.

        Arguments:
            pauli: A Pauli string.
            qubits: A set of qubits.

        Returns:
            The provided Pauli string acting only on the provided qubits.
            Note: This could potentially be empty (identity).
        """
        return cirq.PauliString(p.on(q) for q, p in pauli.items() if q in qubits)


In [47]:
def get_num_qubits(hamiltonian: cirq.PauliSum) -> int:
        return len(hamiltonian.qubits)

In [48]:
def get_bit(value, bit):
        return value >> bit & 1


In [49]:
data_directory: str = "/home/siyuan/LBNL/projects/HamPerf/"
extension: str = ".hdf5"

In [69]:
# 'H2': '/ham_BK-4',
"""
hamiltonian = read_openfermion_hdf5(
        os.path.join(data_directory, 'H2.hdf5'),
        # change key indices we have different encoding ways
        get_hdf5_keys(os.path.join(data_directory, 'H2.hdf5'))[2].rstrip(","),
        # Or # fnames_encodings[fname]
)

data = extract_dataset_hdf5(f"/home/siyuan/LBNL/projects/HamPerf/H2.hdf5", f"/ham_BK-4")

# Get the Hamiltonian operator as SparsePauliOp and its size from the data
ham_op, num_qubits = process_data(data)
"""

print(ham_op)

SparsePauliOp(['IIII', 'XZXI', 'XZXZ', 'YZYI', 'YZYZ', 'ZIII', 'ZZII', 'ZZZI', 'ZZZZ', 'ZIZI', 'ZIZZ', 'IZII', 'IZZZ', 'IZIZ', 'IIZI'],
              coeffs=[-0.44779758+0.j,  0.0140341 +0.j,  0.0140341 +0.j,  0.0140341 +0.j,
  0.0140341 +0.j,  0.28235089+0.j,  0.28235089+0.j,  0.09615022+0.j,
  0.09615022+0.j,  0.08211612+0.j,  0.08211612+0.j,  0.16462321+0.j,
 -0.00398675+0.j,  0.08366739+0.j, -0.00398675+0.j])


In [70]:
pauli_coeffs = {}

for p, c in zip(ham_op.paulis, ham_op.coeffs):
        pauli_coeffs[p.settings['data']] = c

# Create ansatz for VQE
hamiltonian = preprocess_hamiltonian(hamiltonian, drop_term_if=[lambda term: term == ()])
nqubits = get_num_qubits(hamiltonian)
qubits = sorted(set(hamiltonian.qubits))
nterms = len(hamiltonian)

print(f"Hamiltonian has {nterms} term(s) and acts on {nqubits} qubit(s).")

Hamiltonian has 14 term(s) and acts on 4 qubit(s).


In [71]:
grouping_algorithms = {
        # 0: 'no grouping',  ## TEST
        # 1: "1-qubit-wise commuting",
        # nqubits // 4: f"{nqubits // 4}-qubit-wise commuting",
        # nqubits // 2: f"{nqubits // 2}-qubit-wise commuting",
        # 3 * nqubits // 4: f"{3 * nqubits // 4}-qubit-wise commuting",
        nqubits: "Fully commuting",
}
metric_groups = {
        label: (compute_groups(k), k, qubits) for k, label in grouping_algorithms.items()
}

observables_old_k = {}
for label, groups in metric_groups.items():
     observables_old_k[label] = observables_old(groups[0], groups[2])
            
# meas_circuits = {}
# for label, groups in metric_groups.items():
#         print('label:', label)
#         meas_circuits[label] =  measurement_circuit(groups[0], groups[1], groups[2])

In [72]:
def pauli_string_to_string(pauli_string, qubits):
    """Convert a PauliString to a string representation with 'I' for identity."""
    result = []
    for qubit in qubits:
        if qubit in pauli_string.keys():
            result.append(str(pauli_string[qubit]))
        else:
            result.append('-')
    return result

In [73]:
from qiskit.quantum_info import Pauli

qubits = []
for i in range(nqubits):
    qubit = cirq.LineQubit(i)
    qubits.append(qubit)

groups_methods = grouping_algorithms.values()

group_strings_k = {}

for group_method in groups_methods:
    groups_string = []    
    identity_add_flag = False
    for groups in metric_groups['Fully commuting'][0]:
            group_string = []
            for pauli_string_cirq in groups:
                # for pauli_string_cirq in group:
                pauli_string = pauli_string_to_string(pauli_string_cirq, qubits)
                group_string.append(pauli_string)
            groups_string.append(group_string)
            # add identity to the first partition if it's in the Hamiltonian
            identity_key = ''.join('I' for _ in range(num_qubits))
            if Pauli(identity_key) in ham_op._pauli_list and identity_add_flag is False:
                groups_string[0].append(['-']*num_qubits)
                identity_add_flag = True
    group_strings_k[group_method] = groups_string

In [74]:
group_strings_k

{'Fully commuting': [[['Z', '-', '-', '-'],
   ['Z', 'Z', '-', '-'],
   ['-', 'Z', '-', '-'],
   ['Z', 'Z', 'Z', '-'],
   ['Z', 'Z', 'Z', 'Z'],
   ['-', 'Z', '-', 'Z'],
   ['Z', '-', 'Z', '-'],
   ['Z', '-', 'Z', 'Z'],
   ['-', 'Z', 'Z', 'Z'],
   ['-', '-', 'Z', '-'],
   ['-', '-', '-', '-']],
  [['X', 'Z', 'X', '-'],
   ['X', 'Z', 'X', 'Z'],
   ['Y', 'Z', 'Y', '-'],
   ['Y', 'Z', 'Y', 'Z']]]}

## Functions to generate measurement circuits

In [75]:
def diag_w_1q(pstring, n):
    ops = []
    for i in range(n):
        s = pstring[i]
        if s == 'X':
            ops.append(['H', i])
        elif s == 'Y':
            ops.append(['Sdag', i])
            ops.append(['H', i])
            
    return ops


def localize_diagonal(pstring, n):
    indices = []
    for i in range(n):
        if pstring[i] != '-':
            indices.append(i)
            
    if len(indices)<2:
        return [],n
            
    ops = []
    for i in range(len(indices)-1):
        ops.append(['CX', indices[i], indices[i+1]])
     
    return ops, indices[-1]
    

def local_diagonalize(pstring, n):
    
    ops1 = diag_w_1q(pstring,n)
    ops2, rightmost = localize_diagonal(pstring, n) 
    
    return  ops1 + ops2, rightmost

In [76]:
def h(pstring, index):
    
    result = pstring.copy()
    signchange = False
    
    if pstring[index] == 'X':
        result[index] = 'Z'
    elif pstring[index] == 'Y':
        signchange = True
    elif pstring[index] == 'Z':
        result[index] = 'X'
        
    return result, signchange

def s(pstring, index):
    result = pstring.copy()
    signchange = False
    
    if pstring[index] == 'X':
        result[index] = 'Y'
    elif pstring[index] == 'Y':
        result[index] = 'X'
        signchange = True
        
    return result, signchange

def sdag(pstring, index):
    result = pstring.copy()
    signchange = False
    
    if pstring[index] == 'X':
        result[index] = 'Y'
        signchange = True
    elif pstring[index] == 'Y':
        result[index] = 'X'
        
    return result, signchange


def cx(pstring, i1, i2):
    result = pstring.copy()
    signchange = False
    
    p1 = pstring[i1]
    p2 = pstring[i2]
    
    if p1 == '-' and (p2 == 'Z' or p2 == 'Y'):
        result[i1] = 'Z'
    elif p1 == 'Z' and (p2 == 'Z' or p2 == 'Y'):
        result[i1] = '-'
    elif p1 == 'X' and p2 == '-' :
        result[i2] = 'X'
    elif p1 == 'X' and p2 == 'X' :
        result[i2] = '-'
    elif p1 == 'X' and p2 == 'Y' :
        result[i1] = 'Y'
        result[i2] = 'Z'
    elif p1 == 'X' and p2 == 'Z' :
        result[i1] = 'Y'
        result[i2] = 'Y'
        signchange = True
    elif p1 == 'Y' and p2 == '-' :
        result[i2] = 'X'
    elif p1 == 'Y' and p2 == 'X' :
        result[i2] = '-'
    elif p1 == 'Y' and p2 == 'Y' :
        result[i1] = 'X'
        result[i2] = 'Z'
        signchange = True
    elif p1 == 'Y' and p2 == 'Z' :
        result[i1] = 'X'
        result[i2] = 'Y'
        
    return result, signchange

def apply_ops(ops, pstring):
    
    signchange = 1
    result = pstring.copy()
    
    for o in ops:
        
        change = False
        if o[0] == 'H':
            result, change = h(result, o[1])
        elif o[0] == 'S':
            result, change = s(result, o[1])
        elif o[0] == 'Sdag':
            result, change = sdag(result, o[1])
        elif o[0] == 'CX':
            result, change = cx(result, o[1], o[2])
            
        if change:
            signchange = signchange * (-1)
    
    return result, signchange

In [77]:
def is_diagonal(pstring):
    
    for p in pstring:
        if p == 'X' or p == 'Y':
            return False
    
    return True

def get_right_most(pstrings,n):
    
    vals = np.zeros(len(pstrings))
    
    for ip, p in enumerate(pstrings):
        flag = is_diagonal(p)
        if flag:
            vals[ip] = -1
        else:
            for j in range(n-1,-1,-1):
                if p[j] != '-': 
                    vals[ip] = j
                    break
#     print('-'*20)
#     for i in range(len(pstrings)):
#         print(pstrings[i],'\t',vals[i])
#     print('-'*20)
    return np.argmax(vals)

In [78]:
from functools import reduce

def simultaneously_diagonalize_circuit(old_pstringlist):
    #assume that the list is somehow sorted
    ops = []
    
    n = len(old_pstringlist[0])
    
    qc = QuantumCircuit(n)
    
    pstringlist = old_pstringlist.copy()
    changes = []
    
    for i in range(len(pstringlist)):
        index = get_right_most(pstringlist,n)
        new_ops, n = local_diagonalize(pstringlist[index], n)
        #new_ops, n = local_diagonalize(pstringlist[i], n)
        
        ops = ops + new_ops
        pauli_string_change = []
        for j in range(len(pstringlist)):
            pstringlist[j], change = apply_ops(new_ops, pstringlist[j]) 
            pauli_string_change.append(change)
        # add the final sign of the new pauli string
        changes.append(reduce(lambda x, y: x * y, pauli_string_change))
        
    for i in range(len(ops)-1,-1,-1):
        o = ops[i]
        if o[0] == 'H':
            qc.h(o[1])
        elif o[0] == 'S':
            qc.sdg(o[1])
        elif o[0] == 'Sdag':
            qc.s(o[1])
        elif o[0] == 'CX':
            qc.cx(o[1],o[2])
            
    return qc, pstringlist, changes

In [79]:
I = np.array([[1,0],[0,1]])
X = np.array([[0,1],[1,0]])
Y = np.array([[0,-1j],[1j,0]])
Z = np.array([[1,0],[0,-1]])

H = 1/np.sqrt(2) * np.array([[1,1],[1,-1]])       #X->Z, Y->-Y, Z->X
S = np.array([[1,0],[0,1j]])                      #X->Y, Y->-X, Z->Z
Sdag = np.array([[1,0],[0,-1j]])

CX = np.kron((I+Z)/2,I) + np.kron((I-Z)/2,X)

# change pauli string to matrix
def pstring_to_matrix(pstring):
    
    result = 1
    for p in pstring:
        if p == '-':
            result = np.kron(I,result)
        elif p == 'X':
            result = np.kron(X,result)
        elif p == 'Y':
            result = np.kron(Y,result)
        elif p == 'Z':
            result = np.kron(Z,result)
    return result

In [80]:
# Check if it diagonalize the circuit

import scipy.linalg as scila

qcs_k = {}
changes_k = {}
new_paulistringlists_k = {}
for key, val in group_strings_k.items():
    print('group method:', key)
    qcs = []
    changes = []
    new_paulistringlists = []
    for group in val:
        qc, new_paulistringlist, change = simultaneously_diagonalize_circuit(group)
        qcs.append(qc)
        changes.append(change)
        new_paulistringlists.append(new_paulistringlist)
    qcs_k[key] = qcs
    changes_k[key] = changes
    new_paulistringlists_k[key] = new_paulistringlists


    for qc, pstringlist in zip(qcs, val):
        U = Operator.from_circuit(qc).data
        Udag = np.transpose(np.conjugate(U))
        
        for p in pstringlist:
            m = Udag @ pstring_to_matrix(p) @ U
            print(p,'\t',scila.norm(m - np.diag(np.diag(m))))
        print('--------')

group method: Fully commuting
['Z', '-', '-', '-'] 	 0.0
['Z', 'Z', '-', '-'] 	 0.0
['-', 'Z', '-', '-'] 	 0.0
['Z', 'Z', 'Z', '-'] 	 0.0
['Z', 'Z', 'Z', 'Z'] 	 0.0
['-', 'Z', '-', 'Z'] 	 0.0
['Z', '-', 'Z', '-'] 	 0.0
['Z', '-', 'Z', 'Z'] 	 0.0
['-', 'Z', 'Z', 'Z'] 	 0.0
['-', '-', 'Z', '-'] 	 0.0
['-', '-', '-', '-'] 	 0.0
--------
['X', 'Z', 'X', '-'] 	 1.921425704926269e-31
['X', 'Z', 'X', 'Z'] 	 1.921425704926269e-31
['Y', 'Z', 'Y', '-'] 	 1.921425704926269e-31
['Y', 'Z', 'Y', 'Z'] 	 1.921425704926269e-31
--------


In [81]:
new_paulistringlists_k

{'Fully commuting': [[['Z', '-', '-', '-'],
   ['Z', 'Z', '-', '-'],
   ['-', 'Z', '-', '-'],
   ['Z', 'Z', 'Z', '-'],
   ['Z', 'Z', 'Z', 'Z'],
   ['-', 'Z', '-', 'Z'],
   ['Z', '-', 'Z', '-'],
   ['Z', '-', 'Z', 'Z'],
   ['-', 'Z', 'Z', 'Z'],
   ['-', '-', 'Z', '-'],
   ['-', '-', '-', '-']],
  [['-', 'Z', 'Z', '-'],
   ['-', '-', '-', 'Z'],
   ['-', '-', 'Z', '-'],
   ['-', 'Z', '-', 'Z']]]}

In [82]:
changes_k

{'Fully commuting': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1]]}

In [100]:
# circuits, new_all_obs, old_all_obs  = value
ansatz = EfficientSU2(ham_op.num_qubits).decompose()
ansatz.barrier()

initial_params = np.random.uniform(-np.pi, np.pi, ansatz.num_parameters)

# print(ansatz)
# print('------------------')
# ansatz = QuantumCircuit(4)

for key, circuits in qcs_k.items():
    ansatz_mes_circuits = []
    print(f'---partition method: {key}---')
    for circuit in circuits:
        ansatz_mes = ansatz.copy()
        # for circ in circs:
        ansatz_mes.barrier()
        ansatz_mes.compose(circuit, qubits=list(range(circuit.num_qubits)), inplace=True)
        ansatz_mes.measure_active()
        ansatz_mes_circuits.append(ansatz_mes)

    bound_circuits = [ansatz_mes_circuit.assign_parameters(initial_params) for ansatz_mes_circuit in ansatz_mes_circuits]

    # for circuit in bound_circuits:
    #     # circuit.draw('mpl', fold=-1)
    #     print(circuit)
    #     print('---')
    results = AerSimulator().run(bound_circuits, shots=10000).result().get_counts()
    
    # Calculate the expectation value
    new_all_obs = new_paulistringlists_k[key]
    print('---- new all obs----')
    print(new_all_obs)
    changes = changes_k[key]
    old_all_obs = observables_old_k[key]
    print('-----old all obs-----')
    print(old_all_obs)
    print('-----changes------')
    print(changes)
    
    exp = 0
    for res, new_obs, old_obs, change in zip(results, new_all_obs, old_all_obs, changes):
        print('res', res)
        for new_ob, old_ob, sign in zip(new_obs, old_obs, change):
                old_ob_key = old_ob.__str__()[-len(qubits):]
                new_ob = new_ob[::-1]
                new_obs_string = ''.join(x if x!= '-' else 'I' for x in new_ob)
                exp_change = get_expectation_value(new_obs_string, res) * pauli_coeffs[old_ob_key] * sign
                print(f'key: {old_ob_key}-->{new_obs_string}, coef: {pauli_coeffs[old_ob_key]}, get_exp: {get_expectation_value(new_obs_string, res)}, exp change is {exp_change}')
                exp += exp_change
                print('current exp:', exp)
            

        print('expectation value:', exp)


---partition method: Fully commuting---
---- new all obs----
[[['Z', '-', '-', '-'], ['Z', 'Z', '-', '-'], ['-', 'Z', '-', '-'], ['Z', 'Z', 'Z', '-'], ['Z', 'Z', 'Z', 'Z'], ['-', 'Z', '-', 'Z'], ['Z', '-', 'Z', '-'], ['Z', '-', 'Z', 'Z'], ['-', 'Z', 'Z', 'Z'], ['-', '-', 'Z', '-'], ['-', '-', '-', '-']], [['-', 'Z', 'Z', '-'], ['-', '-', '-', 'Z'], ['-', '-', 'Z', '-'], ['-', 'Z', '-', 'Z']]]
-----old all obs-----
[[cirq.DensePauliString('ZIII', coefficient=(1+0j)), cirq.DensePauliString('ZZII', coefficient=(1+0j)), cirq.DensePauliString('IZII', coefficient=(1+0j)), cirq.DensePauliString('ZZZI', coefficient=(1+0j)), cirq.DensePauliString('ZZZZ', coefficient=(1+0j)), cirq.DensePauliString('IZIZ', coefficient=(1+0j)), cirq.DensePauliString('ZIZI', coefficient=(1+0j)), cirq.DensePauliString('ZIZZ', coefficient=(1+0j)), cirq.DensePauliString('IZZZ', coefficient=(1+0j)), cirq.DensePauliString('IIZI', coefficient=(1+0j)), cirq.DensePauliString('IIII', coefficient=(-0.44779757933958947+0j))],

In [101]:
ham_op

SparsePauliOp(['IIII', 'XZXI', 'XZXZ', 'YZYI', 'YZYZ', 'ZIII', 'ZZII', 'ZZZI', 'ZZZZ', 'ZIZI', 'ZIZZ', 'IZII', 'IZZZ', 'IZIZ', 'IIZI'],
              coeffs=[-0.44779758+0.j,  0.0140341 +0.j,  0.0140341 +0.j,  0.0140341 +0.j,
  0.0140341 +0.j,  0.28235089+0.j,  0.28235089+0.j,  0.09615022+0.j,
  0.09615022+0.j,  0.08211612+0.j,  0.08211612+0.j,  0.16462321+0.j,
 -0.00398675+0.j,  0.08366739+0.j, -0.00398675+0.j])

In [102]:
estimator = Estimator(AerSimulator(), options={"default_shots": 10000})

pub = (ansatz, [ham_op], [initial_params])
# pub = (ansatz, [ham_op], [])

result = estimator.run(pubs=[pub]).result()

# Get results for the first (and only) PUB
energy = result[0].data.evs[0]

print(f'expectation value calculated by qiskit: {energy}')

expectation value calculated by qiskit: -0.16870211705022603
